In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency


In [2]:
df_ab = pd.read_csv('./airflow_significance/significance_all_areas.csv')

In [3]:
df_ab.dropna(subset=['area_name'])

,area_name,variant,session_count,converts,Non-converts,CVR
0,Jahra - Saad Al Abdullah,Variation1,55816,3125,52691,5.60%
1,Salmiya,Variation3,55577,4509,51068,8.11%
2,Salmiya,Variation2,55539,4711,50828,8.48%
3,Salmiya,Variation4,55353,4490,50863,8.11%
4,Jahra - Saad Al Abdullah,Variation3,55242,2946,52296,5.33%
...,...,...,...,...,...,...
824,South Abdullah Al Mubarak,Variation4,12,0,12,0.00%
825,South Abdullah Al Mubarak,Variation1,10,0,10,0.00%
826,South Abdullah Al Mubarak,Variation3,10,0,10,0.00%
827,South Abdullah Al Mubarak,Control,9,0,9,0.00%


In [4]:
var=df_ab['variant'].drop_duplicates()
print(var)

0    Variation1
1    Variation3
2    Variation2
3    Variation4
5       Control
9    Variation5
Name: variant, dtype: object


In [5]:
var=var.drop(5)
print(var)

0    Variation1
1    Variation3
2    Variation2
3    Variation4
9    Variation5
Name: variant, dtype: object


In [6]:
myorder = [0, 2, 1, 3, 9]
var = [var[i] for i in myorder]
print(var)

['Variation1', 'Variation2', 'Variation3', 'Variation4', 'Variation5']


In [7]:
df_ab=df_ab.sort_values(['session_count','area_name','variant'],ascending=False)
df_ab.head()

,area_name,variant,session_count,converts,Non-converts,CVR
0,Jahra - Saad Al Abdullah,Variation1,55816,3125,52691,5.60%
1,Salmiya,Variation3,55577,4509,51068,8.11%
2,Salmiya,Variation2,55539,4711,50828,8.48%
3,Salmiya,Variation4,55353,4490,50863,8.11%
4,Jahra - Saad Al Abdullah,Variation3,55242,2946,52296,5.33%


In [8]:
df_ab.rename(columns = {'Non-converts':'n_converts'}, inplace = True)


In [9]:
#t=np.array([])
area=df_ab['area_name'].drop_duplicates()
area=area.dropna()
#for x in area:
   # for y in var:
      # print(x,df_ab['variant'].loc[df_ab['variant']==y])
     # print(x,df_ab.converts.loc[df_ab['variant']=='Control'], df_ab.n_converts.loc[df_ab['variant']==y])
     #   t.append(x)
      #  t= np.vstack(np.array([[df_ab.converts.loc[df_ab['variant']=='Control'],df_ab.n_converts.loc[df_ab['variant']=='Control']], [df_ab.converts.loc[df_ab['variant']==y] ,df_ab.n_converts.loc[df_ab['variant']==y]]]))
      #  print(t)

In [18]:
var

['Variation1', 'Variation2', 'Variation3', 'Variation4', 'Variation5']

In [58]:
p_values = []
for ar in area:
    a=df_ab[df_ab['area_name']==ar]
    p_values=np.append(p_values,ar)
    for v in var:
        T = np.array([[a.converts.loc[a.variant=='Control'], a.n_converts.loc[a.variant=='Control']], [a.converts.loc[a.variant==v], a.n_converts.loc[a.variant==v]]])
        p1 = scipy.stats.chi2_contingency(T,correction=False)[1]
        p_values=np.append(p_values,p1)
      #  p_values.append(scipy.stats.chi2_contingency(T,correction=False)[1])
#table = list(ar, p_values)
#df2 = pd.DataFrame(table, columns=['test_name', 'p_value'])
        #df['kpi_label'] = column_name


ValueError: The internally computed table of expected frequencies has a zero element at (0, 0, 0).

In [60]:
#appending 0s to make the array reshapable. Ideally delete the last entry [0 implies significance.]
for i in range(5):
    p_values=p_values=np.append(p_values,0)

In [61]:
p_values.shape

(822,)

In [62]:
p_values=p_values.reshape(-1,6)

In [63]:
df_results=pd.DataFrame(p_values,columns=['area','Variation1','Variation2','Variation3','Variation4','Variation5'])

In [64]:
df_results.dtypes

area          object
Variation1    object
Variation2    object
Variation3    object
Variation4    object
Variation5    object
dtype: object

In [65]:
convert_dict = {'area':object,
                'Variation1':float,
                'Variation2':float,
                'Variation3':float,
                'Variation4':float,
                'Variation5':float,
                }
df_results = df_results.astype(convert_dict)

In [66]:
df_results.head()

,area,Variation1,Variation2,Variation3,Variation4,Variation5
0,Jahra - Saad Al Abdullah,6.038871e-02,8.312423e-02,0.944058,0.002546,1.738087e-01
1,Salmiya,1.446080e-06,1.097790e-03,0.292990,0.297638,7.994589e-10
2,Sabah Al-Salem,1.857201e-10,1.722242e-07,0.000458,0.105765,2.381524e-02
3,Jaber Al Ahmad,7.623612e-02,3.544127e-03,0.009858,0.033929,7.374618e-01
4,Salwa,2.904787e-03,8.300491e-04,0.012842,0.070132,5.726219e-02


In [70]:
#bonferroni's correction
df_results[df_results.select_dtypes(include=['number']).columns]*=5
df_results.head()

,area,Variation1,Variation2,Variation3,Variation4,Variation5
0,Jahra - Saad Al Abdullah,3.019436e-01,4.156211e-01,4.720292,0.012729,8.690435e-01
1,Salmiya,7.230401e-06,5.488949e-03,1.464952,1.488191,3.997295e-09
2,Sabah Al-Salem,9.286003e-10,8.611209e-07,0.002288,0.528827,1.190762e-01
3,Jaber Al Ahmad,3.811806e-01,1.772064e-02,0.049290,0.169643,3.687309e+00
4,Salwa,1.452394e-02,4.150246e-03,0.064208,0.350662,2.863110e-01


In [71]:
from pathlib import Path  
filepath = Path('./airflow_significance/chi2_values.csv')  
#filepath.parent.mkdir(parents=True, exist_ok=True)  
df_results.to_csv(filepath)  
#area.to_csv(index=True)